In [11]:
NOTEBOOK = True
DATA_DIRPATH = '/aip/datasets/a3/preprocessed/07-02-23/yolo-formated/origin/detect'
DATA_DIRPATH = '/aip/datasets/deel/processed/origin/detect'
CHECKPOINT_DIRPATH = './checkpoints'
EXPERIMENT = "yolo_nas_s_lard"
MODEL = "yolo_nas_s"
INPUT = [2048, 2048]
BATCH_SIZE = 64

In [5]:
# %load_ext autoreload
# %autoreload 2

import sys
sys.path.append('./src')

In [9]:
import copy
import super_gradients

from super_gradients.training import models
from super_gradients.training import Trainer

from pathlib import Path

from torchinfo import summary

from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val
from super_gradients.training.utils.distributed_training_utils import setup_device, MultiGPUMode

In [16]:
setup_device(multi_gpu=MultiGPUMode.DISTRIBUTED_DATA_PARALLEL, num_gpus=8)

/home/aws_install/miniconda3/envs/super-gradients/lib/python3.10/site-packages/traitlets/traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/home/aws_install/miniconda3/envs/super-gradients/lib/python3.10/site-packages/traitlets/traitlets.py:2499: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use '2e5020b4-bf76-4437-9c7b-2ce89cc96507' instead of 'b"2e5020b4-bf76-4437-9c7b-2ce89cc96507"'.
  warn(
/home/aws_install/miniconda3/envs/super-gradients/lib/python3.10/site-packages/traitlets/traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/home/aws_install/miniconda3/envs/super-gradients/lib/python3.10/site-packages/traitlets/traitlets.py:2499: FutureWarning: Supporting

ChildFailedError: 
============================================================
/home/aws_install/miniconda3/envs/super-gradients/lib/python3.10/site-packages/ipykernel_launcher.py FAILED
------------------------------------------------------------
Failures:
[1]:
  time      : 2023-05-12_11:54:07
  host      : ip-10-240-5-73.eu-west-1.compute.internal
  rank      : 1 (local_rank: 1)
  exitcode  : 1 (pid: 106933)
  error_file: <N/A>
  traceback : To enable traceback see: https://pytorch.org/docs/stable/elastic/errors.html
[2]:
  time      : 2023-05-12_11:54:07
  host      : ip-10-240-5-73.eu-west-1.compute.internal
  rank      : 2 (local_rank: 2)
  exitcode  : 1 (pid: 106934)
  error_file: <N/A>
  traceback : To enable traceback see: https://pytorch.org/docs/stable/elastic/errors.html
[3]:
  time      : 2023-05-12_11:54:07
  host      : ip-10-240-5-73.eu-west-1.compute.internal
  rank      : 3 (local_rank: 3)
  exitcode  : 1 (pid: 106935)
  error_file: <N/A>
  traceback : To enable trac

E0512 11:54:07.836300 139646303588544 api.py:672] failed (exitcode: 1) local_rank: 0 (pid: 106932) of binary: /home/aws_install/miniconda3/envs/super-gradients/bin/python


In [ ]:
# model = models.get("yolo_nas_s", pretrained_weights="coco") # s, m or l
model = models.get(EXPERIMENT, num_classes=1) # s, m or l

print(summary(model=model, 
        input_size=(1, 3, *INPUT),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
))


trainer = Trainer(experiment_name=EXPERIMENT, ckpt_root_dir=CHECKPOINT_DIRPATH)

In [17]:
dataset_params = {
    'data_dir': DATA_DIRPATH,
    'train_images_dir':'images/train',
    'train_labels_dir':'labels/train',
    'train_cache_dir': DATA_DIRPATH+'/cache/train',
    'val_images_dir':'images/valid',
    'val_labels_dir':'labels/valid',
    'val_cache_dir': DATA_DIRPATH+'/cache/valid',
    'test_images_dir':'images/valid',
    'test_labels_dir':'labels/valid',
    'test_cache_dir': DATA_DIRPATH+'/cache/valid',
    'classes': ['Runway'],
    'input_dim': INPUT,
    'cache_annotations': True
}

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes'],
        'input_dim': dataset_params['input_dim'],
        'cache_dir': dataset_params['train_cache_dir'],
        'cache_annotations': dataset_params['cache_annotations']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers':8,
        # 'sampler': {"DistributedSampler": {}}
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes'],
        'input_dim': dataset_params['input_dim'],
        'cache_dir': dataset_params['val_cache_dir'],
        'cache_annotations': dataset_params['cache_annotations']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':8,
        'sampler': {"DistributedSampler": {}}
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes'],
        'input_dim': dataset_params['input_dim'],
        'cache_dir': dataset_params['val_cache_dir'],
        'cache_annotations': dataset_params['cache_annotations']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':8,
        'sampler': {"DistributedSampler": {}}
    }
)

{'DetectionMosaic': {'input_dim': [640, 640], 'prob': 1.0}}
{'DetectionRandomAffine': {'degrees': 10.0, 'translate': 0.1, 'scales': [0.1, 2], 'shear': 2.0, 'target_size': [640, 640], 'filter_box_candidates': True, 'wh_thr': 2, 'area_thr': 0.1, 'ar_thr': 20}}
{'DetectionMixup': {'input_dim': [640, 640], 'mixup_scale': [0.5, 1.5], 'prob': 1.0, 'flip_prob': 0.5}}
{'DetectionHSV': {'prob': 1.0, 'hgain': 5, 'sgain': 30, 'vgain': 30}}
{'DetectionHorizontalFlip': {'prob': 0.5}}
{'DetectionPaddedRescale': {'input_dim': [640, 640], 'max_targets': 120}}
{'DetectionTargetsFormatTransform': {'input_dim': [640, 640], 'output_format': 'LABEL_CXCYWH'}}


RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.

{'DetectionPaddedRescale': {'input_dim': [640, 640]}}
{'DetectionTargetsFormatTransform': {'max_targets': 50, 'input_dim': [640, 640], 'output_format': 'LABEL_CXCYWH'}}


: 

In [ ]:
print(train_data.dataset.transforms)

In [ ]:
# Plot a batch
if NOTEBOOK:
  train_data.dataset.plot()

In [ ]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False, # True: do not slow down in notebook
    "average_best_models": True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_steps": 1,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": 100,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=1,
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=1,
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=200,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

trainer.train(model=model, 
              training_params=train_params, 
              train_loader=train_data, 
              valid_loader=val_data)

In [13]:
best_model = models.get(MODEL, num_classes=1, checkpoint_path=Path(f"{CHECKPOINT_DIRPATH}/{EXPERIMENT}/ckpt_best.pth").resolve().as_posix()).cuda()

ReverseImageChannels
{'DetectionLongestMaxSizeRescale': {'output_shape': [2048, 2048]}}
{'DetectionLongestMaxSizeRescale': {'output_shape': [640, 640]}}
{'DetectionBottomRightPadding': {'output_shape': [640, 640], 'pad_value': 114}}
{'ImagePermute': {'permutation': (2, 0, 1)}}


In [ ]:
res = best_model.predict("/efs/players/to122838/resources/landing/videos/paphos.mp4")

In [ ]:
res.save(f"./tmp/{EXPERIMENT}-paphos.mp4")

: 

In [14]:
from super_gradients.training.utils.quantization.selective_quantization_utils import SelectiveQuantizer

q_util = SelectiveQuantizer(
    default_quant_modules_calibrator_weights="max",
    default_quant_modules_calibrator_inputs="histogram",
    default_per_channel_quant_weights=True,
    default_learn_amax=False,
    verbose=True,
)
q_util.quantize_module(best_model)

from super_gradients.training.utils.quantization.calibrator import QuantizationCalibrator

calibrator = QuantizationCalibrator(verbose=True)
calibrator.calibrate_model(
    model,
    method="percentile",
    calib_data_loader=val_dataloader,
    num_calib_batches=16,
    percentile=99.99,
)